# ECDH

In [147]:
import Crypto
from Crypto import *
from Crypto.Util import *

# Private keys
Alice_priv = Crypto.Util.number.getPrime(16)
Bob_priv = Crypto.Util.number.getPrime(16)

# The curve is y^2 = x^3 + ax + b
a = 1234577
b = 3213242
M = 7654319

P = (5234568, 2287747) # A point in the curve

def add(A,B):
    if A==(0,0): return B
    if B==(0,0): return A
    x1,y1 = A
    x2,y2 = B
    if A!=B:
        p = (y2-y1)*pow(x2-x1,M-2,M)
    else:
        p = (3*x1*x1+a)*pow(2*y1,M-2,M)
    x3 = p*p-x1-x2
    y3 = p*(x1-x3)-y1
    return (x3%M,y3%M)

def mult(m, P):
    added = P
    for i in range(m):
        added = add(added,P)
    return added

In [148]:
# Public: P, a, b, M, Bob_pub, Alice_pub
Bob_pub = mult(Bob_priv,P)
Alice_pub = mult(Alice_priv, P)

# Private
shared_secret = mult(Alice_priv, Bob_pub)

# Encryption

In [149]:
# We use the X coordinate of the shared_secret to encrypt.
key = str(shared_secret[0])


import base64
import hashlib
from Crypto import Random
from Crypto.Cipher import AES

class AESCipher(object):

    def __init__(self, key): 
        self.bs = AES.block_size
        self.key = hashlib.sha256(key.encode()).digest()

    def encrypt(self, raw):
        raw = self._pad(raw)
        iv = Random.new().read(AES.block_size)
        cipher = AES.new(self.key, AES.MODE_CBC, iv)
        return base64.b64encode(iv + cipher.encrypt(raw.encode()))

    def decrypt(self, enc):
        enc = base64.b64decode(enc.encode())
        iv = enc[:AES.block_size]
        cipher = AES.new(self.key, AES.MODE_CBC, iv)
        return self._unpad(cipher.decrypt(enc[AES.block_size:])).decode('utf-8')

    def _pad(self, s):
        return s + (self.bs - len(s) % self.bs) * chr(self.bs - len(s) % self.bs)

    @staticmethod
    def _unpad(s):
        return s[:-ord(s[len(s)-1:])]


In [150]:
aes_engine = AESCipher(key)
cipher = aes_engine.encrypt("HACKMADRID{DontUseWeakKeys!}")
print('Encrypted message: ' + str(cipher))

Encrypted message: b'J31ueow1irdvyJ+1aGszetQAozbFFfK8q9hmT4ygOvytezpzJWnEE0eDjVAh5Loe'


# Solucion

In [151]:
# Como enunciado nos dan:
# The curve is y^2 = x^3 + ax + b

a = 1234577
b = 3213242
M = 7654319

P = (5234568, 2287747) # A point in the curve

# Ademas, conocemos las claves publicas de Alice y de Bob, por lo que suponemos un
# intercambio de claves ECDH

# Puede que no sepamos como funciona el algebra de curvas elipticas, pero no hay que asustarse. 
# Nos dan las operaciones hechas

def add(A,B):
    if A==(0,0): return B
    if B==(0,0): return A
    x1,y1 = A
    x2,y2 = B
    if A!=B:
        p = (y2-y1)*pow(x2-x1,M-2,M)
    else:
        p = (3*x1*x1+a)*pow(2*y1,M-2,M)
    x3 = p*p-x1-x2
    y3 = p*(x1-x3)-y1
    return (x3%M,y3%M)

def mult(m, P):
    added = P
    for i in range(m):
        added = add(added,P)
    return added

# Sabemos entonces que las claves privadas son enteros aleatorios de 16 bits de longitud
# Private keys
Alice_priv = Crypto.Util.number.getPrime(16)
Bob_priv = Crypto.Util.number.getPrime(16)

### Por otro lado, nos dan lo que parece ser un texto cifrado en formato base64. Mirando el codigo filtrado del algoritmo, nos damos cuenta de que han usado el secreto compartido ECDH para cifrar un mensaje en AES256.

### Como las claves privadas y el modulo M son relativamente cortos, podemos hacer fuerza bruta para obtener las claves privadas y, de esa forma, el shared_secret que se ha usado para cifrar AES

In [164]:
# Hacemos fuerza bruta para sacar la clave privada
def bruteforce(P, pubkey):
    X = (0,0)
    for i in range(M):
        X = add(X, P)
        if X==pubkey:
            secret = i
            print('Private Key: '+str(secret))
            return secret

# Obtenemos el shared_secret
def getSecret(broken_priv, pub2):
    shared_secret = mult(broken_priv, pub2)
    print('Shared Secret: ' + str(shared_secret))
    return shared_secret

In [165]:
secret = bruteforce(P, Alice_pub)
shared_secret = getSecret(secret, Bob_pub)

Private Key: 53047
Shared Secret: (3128418, 2951723)


### La clave usada en el algoritmo AES del codigo sera el hash SHA256 de la primera coordenada de shared_secret

In [166]:
import AES

encrypted_message = cipher.decode()
key = str(shared_secret[0])
aes_engine = AES.AESCipher(key)
print(aes_engine.decrypt(encrypted_message))

HACKMADRID{DontUseWeakKeys!}
